## CNN: Fine-Tunning com Wisard

### Imports

In [3]:
import wisardpkg as wp
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np
from modules.termometro import codificador_termometro

In [2]:
features = np.load('features.npy', mmap_mode='r')
labels = np.load('labels.npy', mmap_mode='r')
print(features.shape)

(5000, 512)


In [4]:
n_bits = 16
X_encoded_parts = []

# Aplica termômetro para cada feature
for i in range(features.shape[1]):
    coluna = features[:, i]
    coded = codificador_termometro(coluna, n_bits)
    X_encoded_parts.append(coded)

# Concatena todas as colunas codificadas horizontalmente
X_final_binary = np.hstack(X_encoded_parts).astype(np.uint8)


In [5]:
# Se os rótulos não estiverem em string ainda:
y_str = labels.astype(str).tolist()

print(f"\nPrimeiros 5 rótulos de y (como strings): {y_str[:5]}")

X_encoded_uint8 = X_final_binary.astype(np.uint8)

print(f"\nDimensão de X após todas as codificações: {X_encoded_uint8.shape}")
print(f"Tipo dos elementos de X_final_binary: {X_encoded_uint8.dtype}")
print("Primeiras 2 linhas de X_final_binary:\n", X_encoded_uint8[:2])



Primeiros 5 rótulos de y (como strings): ['1', '0', '0', '0', '1']

Dimensão de X após todas as codificações: (5000, 8192)
Tipo dos elementos de X_final_binary: uint8
Primeiras 2 linhas de X_final_binary:
 [[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]]


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X_final_binary, y_str, test_size=0.3, random_state=42, stratify=y_str
)

print(f"Tamanho do conjunto de treino (X_train): {X_train.shape[0]} amostras, {X_train.shape[1]} bits")
print(f"Tamanho do conjunto de teste (X_test): {X_test.shape[0]} amostras, {X_test.shape[1]} bits")
print(f"Tipo dos elementos de X_train: {X_train.dtype}")

Tamanho do conjunto de treino (X_train): 3500 amostras, 8192 bits
Tamanho do conjunto de teste (X_test): 1500 amostras, 8192 bits
Tipo dos elementos de X_train: uint8


In [8]:
addressSize = 8
wsd = wp.Wisard(addressSize, ignoreZero=False, verbose=True)

print(f"WiSARD inicializado com addressSize={addressSize}")
wsd.train(X_train, y_train)

predictions_str = wsd.classify(X_test)

y_pred_np = np.array(predictions_str)
y_test_np = np.array(y_test) 


WiSARD inicializado com addressSize=8


In [10]:
print(classification_report(
    y_test_np, y_pred_np,
    labels=np.unique(y_test)
))

              precision    recall  f1-score   support

           0       0.84      0.95      0.89       773
           1       0.94      0.81      0.87       727

    accuracy                           0.88      1500
   macro avg       0.89      0.88      0.88      1500
weighted avg       0.89      0.88      0.88      1500

